<a href="https://colab.research.google.com/github/Joseph-Willem-Ricci/5450-final-project/blob/Juan/CIS5450_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIT 5450 Final Project
Joanne Crean, Juan Goleniowski, Joseph Ricci

Before running, be sure to create a new Kaggle API token and save it as 'kaggle.json' in your default Google Drive location (/content/drive/MyDrive/)

In [ ]:
# Installs
!pip install -q kaggle
!pip install pandasql
!pip install sqlalchemy==1.4.46

In [ ]:
# Imports
import pandas as pd
import pandasql as ps
from google.colab import drive
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Setup
drive.mount('/content/drive')
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

In [ ]:
# Download Datasets
# Spotify Top 200 and Viral 50 Charts for 2017 through 2021 Dataset
!!kaggle datasets download -d dhruvildave/spotify-charts

# Spotify Audio Features Datasets
!!kaggle datasets download -d rodolfofigueroa/spotify-12m-songs
!!kaggle datasets download -d muhmores/spotify-top-100-songs-of-20152019
!!kaggle datasets download -d sashankpillai/spotify-top-200-charts-20202021
!!kaggle datasets download -d maharshipandya/-spotify-tracks-dataset
!!kaggle datasets download -d vatsalmavani/spotify-dataset
!!kaggle datasets download -d nandhakumarss/spotify-song-tracks
!!kaggle datasets download -d elemento/music-albums-popularity-prediction

# TikTok Popular Songs Dataset
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2019
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2020
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2021

In [ ]:
# Unzip Datasets
!unzip /content/spotify-charts.zip
!unzip /content/spotify-12m-songs.zip
!unzip /content/spotify-top-100-songs-of-20152019.zip
!unzip /content/spotify-top-200-charts-20202021.zip
!unzip /content/-spotify-tracks-dataset.zip
!unzip /content/spotify-dataset.zip
!unzip /content/spotify-song-tracks.zip
!unzip /content/music-albums-popularity-prediction.zip
!unzip /content/tiktok-popular-songs-2019
!unzip /content/tiktok-popular-songs-2020
!unzip /content/tiktok-popular-songs-2021

In [ ]:
# Clean up directory to save space
!rm sample_data/*
!rm -d sample_data
!rm ./*.zip
!rm sample_solution.csv
!rm ./*.xlsx

In [ ]:
# Read the csv files and save them to pandas dataframes
df_charts = pd.read_csv('charts.csv')
df_song_features_1 = pd.read_csv('tracks_features.csv')
df_song_features_2 = pd.read_csv('Spotify 2010 - 2019 Top 100.csv')
df_song_features_3 = pd.read_csv('spotify_dataset.csv')
df_song_features_4 = pd.read_csv('dataset.csv')
df_song_features_5 = pd.read_csv('data/data.csv')
df_song_features_6 = pd.read_csv('SpotifyFeatures.csv')
df_song_features_7 = pd.read_csv('train.csv')
df_song_features_10 = pd.read_csv('test.csv')
df_tiktok_19 = pd.read_csv('TikTok_songs_2019.csv')
df_tiktok_20 = pd.read_csv('TikTok_songs_2020.csv')
df_tiktok_21 = pd.read_csv('TikTok_songs_2021.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clean up remaining files
!rm data/*
!rm -d data
!rm ./*.csv

# Spotify Top 200 and Viral 50 Charts EDA

In [ ]:
# Create a separate column for Year
df_charts['Year']=df_charts['date'].str.slice(stop=4)

# Cast 'Year' as Int
df_charts['Year'] = df_charts['Year'].astype(int)

# Show datatypes 
df_charts.dtypes

#remove ID, trend and streams columns
df_charts.drop(['url', 'trend', 'streams'], axis=1, inplace=True)

df_charts.dropna(inplace=True)

In [ ]:
# Number of records per year
df_charts.groupby(by='Year', as_index=False).count()

In [ ]:
# Gets the highes ranking achieved by each song in the period 2017-2018 in any country
max_ranking_df = df_charts[df_charts['Year']<2019].groupby('title').min('rank')


In [ ]:
max_ranking_df

In [ ]:
# Remove records with year 2017 and 2018
df_charts=df_charts[df_charts['Year'] > 2018]

In [ ]:
# Add max 2017-2018 ranking to df_charts
df_charts=df_charts.merge(max_ranking_df[['rank']], left_on=['title'], right_on=['title'], how='left')

In [ ]:
# Rename Rank columns
df_charts.rename(columns={'rank_x':'rank', 'rank_y':'Best_2017-28'}, inplace=True)

In [ ]:
# Replace NaN with 0
df_charts['Best_2017-28']=df_charts['Best_2017-28'].fillna(0)

In [ ]:
# Best ranking in 2017-2018 for songs in the 2019-2021 group
df_charts

In [ ]:
# Top 50 artists in the 2019-2021 period by number of days in the top position

query = """SELECT artist, count(date) as 'days as number 1'
FROM df_charts
WHERE rank == 1
GROUP BY artist
ORDER BY count(date) DESC
LIMIT 50
"""

sql_top_50_artists = ps.sqldf(query, locals())

In [ ]:
sql_top_50_artists

In [ ]:
# Top 50 songs in the 2019-2021 period by number of days in the top position

query = """SELECT title, count(date) as 'days as number 1'
FROM df_charts
WHERE rank == 1
GROUP BY title
ORDER BY count(date) DESC
LIMIT 50
"""

sql_top_50_songs = ps.sqldf(query, locals())

In [ ]:
sql_top_50_songs

In [ ]:
# Top 50 longest trending songs in the US in the 2019-2021 period

query = """SELECT title, count(date) as 'days trending'
FROM df_charts
WHERE region == 'United States'
GROUP BY title
ORDER BY count(date) DESC
LIMIT 50
"""

sql_top_50_longest_trending_songs = ps.sqldf(query, locals())

In [ ]:
sql_top_50_longest_trending_songs

In [ ]:
# Top 50 longest trending songs outside of the US in the 2019-2021 period

query = """SELECT title, count(date) as 'days trending'
FROM df_charts
WHERE region != 'United States'
GROUP BY title
ORDER BY count(date) DESC
LIMIT 50
"""

sql_top_50_longest_trending_ROW_songs = ps.sqldf(query, locals())

In [ ]:
sql_top_50_longest_trending_ROW_songs

# Spotify Audio Feature Dataset EDA

Since the Spotify Top 200 and Viral 50 charts dataset does not come with audio features the songs, there is no guarantee that we can find datasets that include audio features for all of the songs in the charts. One relevant question to explore is therefore

1. Can we find audio feature data for a suitably "big" fraction of the songs in the Charts dataset?

Additionally, since we'll be combining multiple audio features datasets, there is no guarantee that those datasets share the same features, so

2. What fields do the datasets share?

In [ ]:
# First, since we want to find song features for as many of the songs in 'df_charts' as possible, let's start by
# projecting df_charts by song title and artist name, which will uniquely identify each song, and then dropping duplicates
df_charts_songs_artists = df_charts[['title', 'artist']].drop_duplicates()

# Next, let's join the features datasets on song title and artist name and see what percentage of songs in charts_df
# each song features dataset can provide features for

# Clean extranneous characters from artist columns if necessary
df_song_features_1['artists'] = df_song_features_1['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_5['artists'] = df_song_features_5['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_7['artists'] = df_song_features_7['artists'].str.replace(', ', '')
df_song_features_10['artists'] = df_song_features_10['artists'].str.replace(', ', '')

In [ ]:
# Rename song title "title" and "artist" columns in each dataframe and 
# create separate dataframes for features_7 and features_10, which each have three columns for each field for three different songs
df_song_features_1 = df_song_features_1.rename(columns={'name': 'title', 'artists': 'artist'})
# _song_features_2 already has fields 'title' and 'artist'
df_song_features_3 = df_song_features_3.rename(columns={'Song Name': 'title', 'Artist': 'artist'})
df_song_features_4 = df_song_features_4.rename(columns={'track_name': 'title', 'artists': 'artist'})
df_song_features_5 = df_song_features_5.rename(columns={'name': 'title', 'artists': 'artist'})
df_song_features_6 = df_song_features_6.rename(columns={'track_name': 'title', 'artist_name': 'artist'})
df_song_features_8 = df_song_features_7.rename(columns={'t_name1': 'title', 'artists': 'artist'}).drop(columns=['t_name0', 't_name2', 't_dur0', 't_dur2', 't_dance0', 't_dance2', 't_energy0', 't_energy2', 't_key0', 't_key2', 't_mode0', 't_mode2', 't_speech0', 't_speech2', 't_acous0', 't_acous2', 't_ins0', 't_ins2', 't_live0', 't_live2', 't_val0', 't_val2', 't_tempo0', 't_tempo2', 't_sig0', 't_sig2'])
df_song_features_9 = df_song_features_7.rename(columns={'t_name2': 'title', 'artists': 'artist'}).drop(columns=['t_name0', 't_name1', 't_dur0', 't_dur1', 't_dance0', 't_dance1', 't_energy0', 't_energy1', 't_key0', 't_key1', 't_mode0', 't_mode1', 't_speech0', 't_speech1', 't_acous0', 't_acous1', 't_ins0', 't_ins1', 't_live0', 't_live1', 't_val0', 't_val1', 't_tempo0', 't_tempo1', 't_sig0', 't_sig1'])
df_song_features_7 = df_song_features_7.rename(columns={'t_name0': 'title', 'artists': 'artist'}).drop(columns=['t_name1', 't_name2', 't_dur1', 't_dur2', 't_dance1', 't_dance2', 't_energy1', 't_energy2', 't_key1', 't_key2', 't_mode1', 't_mode2', 't_speech1', 't_speech2', 't_acous1', 't_acous2', 't_ins1', 't_ins2', 't_live1', 't_live2', 't_val1', 't_val2', 't_tempo1', 't_tempo2', 't_sig1', 't_sig2'])
df_song_features_11 = df_song_features_10.rename(columns={'t_name1': 'title', 'artists': 'artist'}).drop(columns=['t_name0', 't_name2', 't_dur0', 't_dur2', 't_dance0', 't_dance2', 't_energy0', 't_energy2', 't_key0', 't_key2', 't_mode0', 't_mode2', 't_speech0', 't_speech2', 't_acous0', 't_acous2', 't_ins0', 't_ins2', 't_live0', 't_live2', 't_val0', 't_val2', 't_tempo0', 't_tempo2', 't_sig0', 't_sig2'])
df_song_features_12 = df_song_features_10.rename(columns={'t_name2': 'title', 'artists': 'artist'}).drop(columns=['t_name0', 't_name1', 't_dur0', 't_dur1', 't_dance0', 't_dance1', 't_energy0', 't_energy1', 't_key0', 't_key1', 't_mode0', 't_mode1', 't_speech0', 't_speech1', 't_acous0', 't_acous1', 't_ins0', 't_ins1', 't_live0', 't_live1', 't_val0', 't_val1', 't_tempo0', 't_tempo1', 't_sig0', 't_sig1'])
df_song_features_10 = df_song_features_10.rename(columns={'t_name0': 'title', 'artists': 'artist'}).drop(columns=['t_name1', 't_name2', 't_dur1', 't_dur2', 't_dance1', 't_dance2', 't_energy1', 't_energy2', 't_key1', 't_key2', 't_mode1', 't_mode2', 't_speech1', 't_speech2', 't_acous1', 't_acous2', 't_ins1', 't_ins2', 't_live1', 't_live2', 't_val1', 't_val2', 't_tempo1', 't_tempo2', 't_sig1', 't_sig2'])

# Project only the necessary columns from each feature dataset and also rename to maintain consistency
features_1_projected = df_song_features_1[['title', 'artist']]
features_2_projected = df_song_features_2[['title', 'artist']]
features_3_projected = df_song_features_3[['title', 'artist']]
features_4_projected = df_song_features_4[['title', 'artist']]
features_5_projected = df_song_features_5[['title', 'artist']]
features_6_projected = df_song_features_6[['title', 'artist']]
features_7_projected = df_song_features_7[['title', 'artist']]
features_8_projected = df_song_features_8[['title', 'artist']]
features_9_projected = df_song_features_9[['title', 'artist']]
features_10_projected = df_song_features_10[['title', 'artist']]
features_11_projected = df_song_features_11[['title', 'artist']]
features_12_projected = df_song_features_12[['title', 'artist']]
all_songs_with_features = pd.concat([features_1_projected, features_2_projected, features_3_projected, features_4_projected, 
                                     features_5_projected, features_6_projected, features_7_projected, features_8_projected, 
                                     features_9_projected, features_10_projected, features_11_projected, features_12_projected], ignore_index=True).drop_duplicates()

# Join on song title and artist name
feature_matches = pd.merge(df_charts_songs_artists, all_songs_with_features, on=['title', 'artist'], how='inner')

# Calculate the percentage of songs each features dataset was able to provide song features for
match_percentage_combined = 100 * feature_matches.shape[0] / df_charts_songs_artists.shape[0]
print("We are able to provide audio features for " + str(round(match_percentage_combined,2)) + "% of songs in df_charts,"\
      " for a total of " + str(feature_matches.shape[0]) + " out of " + str(df_charts_songs_artists.shape[0]) + " songs")

Lets now see whether we can do any additional cleaning to boost our match percentage. Since there are many obscure songs in df_charts that have, say, reached chart position 200 in small countries like Latvia and Ecuador, let us limit our search to songs that charted highly in the United States, and see if any of those songs were unmatched in the features data

In [ ]:
non_matches = pd.merge(df_charts[df_charts['rank'] > 10][df_charts['region'] == 'United States'][['title', 'artist']], 
                                  all_songs_with_features, on=['title', 'artist'], how='left', indicator=True).drop_duplicates()
non_matches = non_matches[non_matches['_merge'] == 'left_only']
non_matches.head()

In [ ]:
# Does all_songs_with_features contain any of these songs but perhaps with a different format for title and artist?
print(all_songs_with_features[all_songs_with_features['artist'] == 'NLE Choppa, Roddy Ricch'])

This suggests that when there is a featured artist, the ordering of the artist names in the features datasets may be the opposite of the ordering in the charts dataset. Lets look at the other row with a featured artist.

In [ ]:
print(all_songs_with_features[all_songs_with_features['artist'] == "Khalid, Summer Walker"])

In this case, the featured artist is not included in the 'artist' column in the charts dataset, but is in the features datasets. In joins so far, we have been joining on 'artist' and 'title' to be sure to uniquely identify each song. But since these songs with featured artists have varying formats in the 'artist' column, but the same format in the 'title' column, and since they are sure to be uniquely identified just by song title, let us try to filter by songs that have "(feat. " in their title, and join the charts and features datasets on 'title'.

In [ ]:
charts_featured = df_charts_songs_artists[df_charts_songs_artists['title'].str.contains("\(feat\. ")][['title', 'artist']]
featured_feature_matches = pd.merge(charts_featured, all_songs_with_features, on=['title'], how='inner').drop_duplicates(subset='title')
featured_feature_matches.drop(columns=['artist_x'], inplace=True)
featured_feature_matches.rename(columns={'artist_y': 'artist'}, inplace=True)
feature_matches = pd.concat([feature_matches, featured_feature_matches], ignore_index=True).drop_duplicates()
match_percentage_combined = 100 * feature_matches.shape[0] / df_charts_songs_artists.shape[0]
print("We are able to provide audio features for " + str(round(match_percentage_combined,2)) + "% of songs in df_charts,"\
      " for a total of " + str(feature_matches.shape[0]) + " out of " + str(df_charts_songs_artists.shape[0]) + " songs")

Next, lets see what columns each of the features datasets share, and create one unified features dataset.

In [ ]:
print([col for col in df_song_features_1.columns])
print([col for col in df_song_features_2.columns])
print([col for col in df_song_features_3.columns])
print([col for col in df_song_features_4.columns])
print([col for col in df_song_features_5.columns])
print([col for col in df_song_features_6.columns])
print([col for col in df_song_features_7.columns])
print([col for col in df_song_features_8.columns])
print([col for col in df_song_features_9.columns])
print([col for col in df_song_features_10.columns])
print([col for col in df_song_features_11.columns])
print([col for col in df_song_features_12.columns])

Common columns:

title, artist, tempo, energy, danceability, liveness, valence, duration, acousticness, speechiness

In [ ]:
features_1_drop_cols = ['loudness', 'album', 'id', 'album_id', 'artist_ids', 'track_number', 'disc_number', 'explicit', 'key', 'mode', 'instrumentalness', 'time_signature', 'year', 'release_date']
features_2_drop_cols = ['dB', 'year released', 'added', 'pop', 'top year', 'artist type', 'top genre']
features_3_drop_cols = ['Chord', 'Loudness', 'Index', 'Highest Charting Position', 'Number of Times Charted', 'Week of Highest Charting', 'Streams', 'Artist Followers', 'Song ID', 'Genre', 'Release Date', 'Weeks Charted', 'Popularity']
features_4_drop_cols = ['loudness', 'Unnamed: 0', 'track_id', 'album_name', 'popularity', 'explicit', 'key', 'mode', 'instrumentalness', 'time_signature', 'track_genre']
features_5_drop_cols = ['loudness', 'year', 'explicit', 'id', 'instrumentalness', 'key', 'mode', 'popularity', 'release_date']
features_6_drop_cols = ['loudness', 'genre', 'track_id', 'popularity', 'instrumentalness', 'key', 'mode', 'time_signature']
features_7_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key0', 't_mode0', 't_ins0', 't_sig0', 'popularity']
features_8_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key1', 't_mode1', 't_ins1', 't_sig1', 'popularity']
features_9_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key2', 't_mode2', 't_ins2', 't_sig2', 'popularity']
features_10_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key0', 't_mode0', 't_ins0', 't_sig0']
features_11_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key1', 't_mode1', 't_ins1', 't_sig1']
features_12_drop_cols = ['id', 'name', 'release_date', 'total_tracks', 't_key2', 't_mode2', 't_ins2', 't_sig2']

features_2_rename_map = {'bpm': 'tempo', 'nrgy': 'energy', 'dnce': 'danceability', 'live': 'liveness', 'val': 'valence', 'dur': 'duration_ms', 'acous': 'acousticness', 'spch': 'speechiness'}
features_3_rename_map = {'Danceability': 'danceability', 'Energy': 'energy', 'Speechiness': 'speechiness', 'Acousticness': 'acousticness', 'Liveness': 'liveness', 'Tempo': 'tempo', 'Duration (ms)': 'duration_ms', 'Valence': 'valence'}
features_7_rename_map = {'t_dance0': 'danceability', 't_energy0': 'energy', 't_speech0': 'speechiness', 't_acous0': 'acousticness', 't_live0': 'liveness', 't_tempo0': 'tempo', 't_dur0': 'duration_ms', 't_val0': 'valence'}
features_8_rename_map = {'t_dance1': 'danceability', 't_energy1': 'energy', 't_speech1': 'speechiness', 't_acous1': 'acousticness', 't_live1': 'liveness', 't_tempo1': 'tempo', 't_dur1': 'duration_ms', 't_val1': 'valence'}
features_9_rename_map = {'t_dance2': 'danceability', 't_energy2': 'energy', 't_speech2': 'speechiness', 't_acous2': 'acousticness', 't_live2': 'liveness', 't_tempo2': 'tempo', 't_dur2': 'duration_ms', 't_val2': 'valence'}
features_10_rename_map = {'t_dance0': 'danceability', 't_energy0': 'energy', 't_speech0': 'speechiness', 't_acous0': 'acousticness', 't_live0': 'liveness', 't_tempo0': 'tempo', 't_dur0': 'duration_ms', 't_val0': 'valence'}
features_11_rename_map = {'t_dance1': 'danceability', 't_energy1': 'energy', 't_speech1': 'speechiness', 't_acous1': 'acousticness', 't_live1': 'liveness', 't_tempo1': 'tempo', 't_dur1': 'duration_ms', 't_val1': 'valence'}
features_12_rename_map = {'t_dance2': 'danceability', 't_energy2': 'energy', 't_speech2': 'speechiness', 't_acous2': 'acousticness', 't_live2': 'liveness', 't_tempo2': 'tempo', 't_dur2': 'duration_ms', 't_val2': 'valence'}

df_song_features_1 = df_song_features_1.drop(columns=features_1_drop_cols)
df_song_features_2 = df_song_features_2.drop(columns=features_2_drop_cols).rename(columns=features_2_rename_map)
df_song_features_3 = df_song_features_3.drop(columns=features_3_drop_cols).rename(columns=features_3_rename_map)
df_song_features_4 = df_song_features_4.drop(columns=features_4_drop_cols)
df_song_features_5 = df_song_features_5.drop(columns=features_5_drop_cols)
df_song_features_6 = df_song_features_6.drop(columns=features_6_drop_cols)
df_song_features_7 = df_song_features_7.drop(columns=features_7_drop_cols).rename(columns=features_7_rename_map)
df_song_features_8 = df_song_features_8.drop(columns=features_8_drop_cols).rename(columns=features_8_rename_map)    
df_song_features_9 = df_song_features_9.drop(columns=features_9_drop_cols).rename(columns=features_9_rename_map)
df_song_features_10 = df_song_features_10.drop(columns=features_10_drop_cols).rename(columns=features_10_rename_map)
df_song_features_11 = df_song_features_11.drop(columns=features_11_drop_cols).rename(columns=features_11_rename_map)
df_song_features_12 = df_song_features_12.drop(columns=features_12_drop_cols).rename(columns=features_12_rename_map)

Check that all features dataframes have the same columns now:

In [ ]:
print(([col for col in df_song_features_1.columns.sort_values()]))
print(([col for col in df_song_features_2.columns.sort_values()]))
print(([col for col in df_song_features_3.columns.sort_values()]))
print(([col for col in df_song_features_4.columns.sort_values()]))
print(([col for col in df_song_features_5.columns.sort_values()]))
print(([col for col in df_song_features_6.columns.sort_values()]))
print(([col for col in df_song_features_7.columns.sort_values()]))
print(([col for col in df_song_features_8.columns.sort_values()])) 
print(([col for col in df_song_features_9.columns.sort_values()]))
print(([col for col in df_song_features_10.columns.sort_values()]))
print(([col for col in df_song_features_11.columns.sort_values()]))
print(([col for col in df_song_features_12.columns.sort_values()]))

Now lets concatenate all of these features dataframes into one and do an inner join with `all_songs_with_features` to get the final, combined features dataset

In [ ]:
df_features = pd.concat([df_song_features_1, df_song_features_2, df_song_features_3, df_song_features_4, df_song_features_5, df_song_features_6, df_song_features_7, df_song_features_8, df_song_features_9, df_song_features_10, df_song_features_11, df_song_features_12], axis=0, ignore_index=True)
df_features = df_features.dropna()
df_features = pd.merge(df_features, feature_matches, on=['title', 'artist'], how='inner')
df_features = df_features.drop_duplicates(subset=['title', 'artist'])
df_features.reset_index(drop=True, inplace=True)
df_features.head(100)  # inspect

In [ ]:
df_features.describe

# TikTok Popular Songs Dataset EDA

In [ ]:
df_tiktok_19.head()

In [ ]:
df_tiktok_20.head()

In [ ]:
df_tiktok_21.head()

In [ ]:
# Combine the tiktok datasets
data_tiktok = [df_tiktok_19, df_tiktok_20, df_tiktok_21]
df_tiktok_full = pd.concat(data_tiktok)

In [ ]:
# Extract the track name and artist to check for the overlap
df_tiktok_tracks = df_tiktok_full[['track_name', 'artist_name']]

# Check number of rows
df_tiktok_tracks.shape[0]

In [ ]:
# Merge with spotify features on track names only to check overlap
df_tiktok_tracks_only = feature_matches.merge(df_tiktok_tracks.drop_duplicates(), left_on = ['title'], right_on=['track_name'], how='inner')

# Check overlap
df_tiktok_tracks_only.shape[0]

In [ ]:
# Review rows that didn't match to see if there's more clean-up that can be done 
df_tiktok_full_merge = feature_matches.merge(df_tiktok_tracks.drop_duplicates(), left_on = ['title'], right_on=['track_name'], how='outer', indicator = True)

In [ ]:
df_tiktok_full_merge.head()

In [ ]:
# Review rows that didn't match to see if there's more clean-up that can be done 
print(df_tiktok_full_merge[df_tiktok_full_merge['_merge'] != 'both'])

In [ ]:
# do a check to see if I convert the track names to lower case, could I get more matches 
df_tiktok_full_merge.dropna(inplace = True)
df_tiktok_full_merge['title_l'] = df_tiktok_full_merge.apply(lambda x : x['title'].lower(), axis =1)
df_tiktok_full_merge['track_name_l'] = df_tiktok_full_merge.apply(lambda x : x['track_name'].lower(), axis =1)

print(df_tiktok_full_merge[(df_tiktok_full_merge['title_l'] == df_tiktok_full_merge['track_name_l']) & (df_tiktok_full_merge['_merge'] != 'both')])

# prints an empty dataframe so the conclusion is that no I can't

In [ ]:
# Create a match column to indicate cases where the tiktok artist name can be found as a substring in the spotify artist column 
df_tiktok_tracks_only['match'] = df_tiktok_tracks_only.apply(lambda x: x['artist_name'].lower().find(x['artist'].lower()), axis=1).ge(0)

# Create a match column to indicate cases where the spotify artist column can be found as a substring in the tiktok artist name 
df_tiktok_tracks_only['match_2'] = df_tiktok_tracks_only.apply(lambda x: x['artist'].lower().find(x['artist_name'].lower()), axis=1).ge(0)

In [ ]:
# review data to see which rows have matched
print(df_tiktok_tracks_only[df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2']])

In [ ]:
# review data to see which rows have not matched
print(df_tiktok_tracks_only[(df_tiktok_tracks_only['match'] == False) & (df_tiktok_tracks_only['match_2'] == False)])

In [ ]:
# check the rows where the artists names aren't an exact match - for visual inspection 
df_tiktok_tracks_only.loc[(df_tiktok_tracks_only['artist'] != df_tiktok_tracks_only['artist_name']) & (df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2'])]

In [ ]:
# Create a dataframe of overlapping songs based on the two matches colums and drop duplicates
df_tiktok_matches_final = df_tiktok_tracks_only.loc[(df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2'])]

# Check the shape of the df 
df_tiktok_matches_final.shape[0]

In [ ]:
# spot check an artist to see if the name matching looks appropriate 
df_tiktok_matches_final.loc[(df_tiktok_matches_final['artist_name'] == "Justin Bieber" )|( df_tiktok_matches_final['artist'] == "Justin Bieber" )]

In [ ]:
# drop rows that are no longer needed
df_tiktok_matches_final.drop(['match', 'match_2', 'track_name', 'artist_name'], inplace =True, axis =1)

# add a column to indicate tiktok popularity 
df_tiktok_matches_final= df_tiktok_matches_final.assign(tiktok_pop = 1)

In [ ]:
df_tiktok_matches_final.head()

In [ ]:
df_tiktok_matches_final.groupby(['tiktok_pop']).count()

# Genius Lyrics Dataset Creation and EDA

In [ ]:
!pip install lyricsgenius

In [ ]:
from google.colab import files
feature_matches.to_csv('combined_features.csv') 
files.download('combined_features.csv')

In [ ]:
import lyricsgenius
import os
import spacy
import re

In [ ]:
client_id = 'TS15U5iwbWLGkhfGFVqnOuDA9mVjJhhLlXpJDYai6nm79S9JWFzznlsQN5dCFuZG'
client_secret = 'SOhrXQxD9YZ2RxBQwR-wwu5Zbxh6UgkfuIEaUJltx9L9h8aynN5zZ9Jsm1JNlh_5Npu_uev1MKorJV_A6MVYKw'
access_token = 'US00oQ6_8lkhwRjHAOudB62bruf1B3JGOGuHR8V8zeawxpoc8fcA9QGXQ3bhYWu-'
website_url = 'https://github.com/Joseph-Willem-Ricci/5450-final-project'

In [ ]:
"""genius = lyricsgenius.Genius(access_token)
artist = genius.search_artist('Andy Shauf')
artist.save_lyrics()"""

In [ ]:
"""api = lyricsgenius.Genius(access_token)
artist = api.artist(380491)
print(artist)"""


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
#stopword_en = nltk.corpus.stopwords.words('english')
#stopword_es = nltk.corpus.stopwords.words('spanish')
#stopword = stopword_en + stopword_es
stopwords = set(stopwords.words(['english', 'spanish', 'german','french']))


In [ ]:
!!kaggle datasets download -d joannecrean/spotify-songs-lyrics  #FIXME: Generalize this to run on anybody's colab
!unzip /content/spotify-songs-lyrics.zip


In [ ]:
df_lyrics = pd.read_csv('all_song_data_complete.csv')

In [ ]:
sensitive_words = pd.read_csv('sensitive_words.csv')
sensitive_words =sensitive_words.values.tolist()


In [ ]:
def tokenize_content(content):
  content = content.lower() #convert to lowercase
  content_tokens = nltk.word_tokenize(content) # tokenise
  alpha_tokens = [w for w in content_tokens if w.isalpha()] # keep alpha tokens
  stop_tokens = [t for t in alpha_tokens if not t in stopwords] #remove if stopword
  final_tokens = [s for s in stop_tokens if not s in sensitive_words] #remove if sensitive
  #print(final_tokens)
  return final_tokens

In [ ]:
df_lyrics.head(10)

In [ ]:
len(df_lyrics)

In [ ]:
df_lyrics.loc[df_lyrics['title'] == 'You Make My Dreams (Come True)']

In [ ]:
df_lyrics.dropna(inplace = True)

In [ ]:
df_lyrics.loc[df_lyrics['song URL'] == 'https://genius.com/Gerald-haywood-2018-haywoodindahood-listening-log-annotated']

In [ ]:
# clean up by dropping rows that aren't lyrics
df_lyrics = df_lyrics[~df_lyrics['song URL'].str.contains('annotated')]

In [ ]:
# remove the intro text before the lyrics 
df_lyrics['lyrics'] = [re.sub('^.*Lyrics', '', x) for x in df_lyrics['lyrics']]

In [ ]:
#remove rows where the URL is not correct so the lyrics are not accurate

df_lyrics['clean URL'] = df_lyrics['song URL'].str.replace('-', '').str.lower()
df_lyrics['clean artist'] = df_lyrics['artist'].str.replace(' ', '').str.lower()

df_lyrics['URL letters'] = df_lyrics['clean URL'].str.extract(r'https://genius\.com/(\w{2})')

#print(df_lyrics.head(10))

df_lyrics = df_lyrics.loc[df_lyrics['URL letters'] == df_lyrics['clean artist'].str[:2]]

In [ ]:
df_lyrics.head(10)

In [ ]:
len(df_lyrics)

In [ ]:
df_lyrics.head(10)

In [ ]:
#df_lyrics_clean = df_lyrics.apply(lambda x: x['Unnamed: 0'] , axis=1)

In [ ]:
df_lyrics['tokenized'] = df_lyrics.apply(lambda x: tokenize_content(x.lyrics), axis=1)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def retrieve_sentiment(content):
  return sia.polarity_scores(content)['compound']


In [ ]:
df_lyrics['sentiment'] = df_lyrics.apply(lambda x: retrieve_sentiment(x.lyrics), axis=1)
df_lyrics_positive = df_lyrics.sort_values(by = ['sentiment'], ascending = False)
df_lyrics_positive.head(50)

In [ ]:
df_lyrics_negative = df_lyrics.sort_values(by = ['sentiment'])
df_lyrics_negative.head(50)

In [ ]:
def add_spacy_data(dataset, feature_column):
    '''
    Grabs the verb, adverb, noun, and stop word Parts of Speech (POS) 
    tokens and pushes them into a new dataset. returns an 
    enriched dataset.
    
    Parameters:
    
    dataset (dataframe): the dataframe to parse
    feature_column (string): the column to parse in the dataset.
    
    Returns: 
    dataframe
    '''
    
    verbs = []
    nouns = []
    adverbs = []
    corpus = []
    nlp = spacy.load('en_core_web_sm')
    ##
    for i in range (0, len(dataset)):
        print("Extracting verbs and topics from record {} of {}".format(i+1, len(dataset)), end = "\r")
        song = dataset.iloc[i][feature_column]
        doc = nlp(song)
        spacy_dataframe = pd.DataFrame()
        for token in doc:
            if token.lemma_ == "-PRON-":
                    lemma = token.text
            else:
                lemma = token.lemma_
            row = {
                "Word": token.text,
                "Lemma": lemma,
                "PoS": token.pos_,
                "Stop Word": token.is_stop
            }
            spacy_dataframe = spacy_dataframe.append(row, ignore_index = True)
        verbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "VERB"].values))
        nouns.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "NOUN"].values))
        adverbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "ADV"].values))
        corpus_clean = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["Stop Word"] == False].values)
        corpus_clean = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_clean)   
        corpus.append(corpus_clean)
    dataset['verbs'] = verbs
    dataset['nouns'] = nouns
    dataset['adverbs'] = adverbs
    dataset['corpus'] = corpus

    return dataset

In [ ]:
# alt spacy dataset
"""
nlp = spacy.load('en_core_web_sm')
    ##
    def extract_spacy_data(song):
        doc = nlp(song)
        spacy_dataframe = pd.DataFrame([{
            "Word": token.text,
            "Lemma": token.lemma_ if token.lemma_ != "-PRON-" else token.text,
            "PoS": token.pos_,
            "Stop Word": token.is_stop
        } for token in doc])
        verbs = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "VERB"].values)
        nouns = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "NOUN"].values)
        adverbs = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "ADV"].values)
        corpus_clean = " ".join([token.lemma_ if token.lemma_ != "-PRON-" else token.text
                                 for token in doc if not token.is_stop])
        corpus_clean = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_clean)
        return verbs, nouns, adverbs, corpus_clean
    
    dataset[['verbs', 'nouns', 'adverbs', 'corpus']] = dataset[feature_column].apply(
        lambda x: pd.Series(extract_spacy_data(x)), axis=1)"""

In [ ]:
"""df_lyrics_words = add_spacy_data(df_lyrics, 'lyrics')"""

In [ ]:
"""df_lyrics_words.head(5)"""

In [ ]:
#df_lyrics_words = df_lyrics_words.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
"""word_counts = []
unique_word_counts = []
for i in range (0, len(df_lyrics_words)):
    word_counts.append(len(df_lyrics_words.iloc[i]['lyrics'].split()))
    unique_word_counts.append(len(set(df_lyrics_words.iloc[i]['lyrics'].split())))
df_lyrics_words['word counts'] = word_counts
df_lyrics_words['unique word counts'] = unique_word_counts"""

In [ ]:
"""df_lyrics_words['common noun']= df_lyrics_words.apply(lambda x: x['nouns'].value_counts().idxmax())
df_lyrics_words['common verb']= df_lyrics_words.apply(lambda x: x['verbs'].value_counts().idxmax())"""

In [ ]:
"""df_lyrics_ready = df_lyrics_words[['title', 'artist', 'sentiment', 'word count', 'unique word count', 'common noun', 'common verb']]"""

# Creating Final Dataset and EDA



In [ ]:
"""df_features_lyrics =feature_matches.merge(df_lyrics_ready, left_on = ['title'], right_on = ['title'], how = 'inner')"""

In [ ]:
"""df_features_lyrics_tiktok = df_features_lyrics.merge(df_tiktok_matches_final, left_on = ['title'], right_on = ['title'], how = 'inner')"""